<a href="https://colab.research.google.com/github/tackulus/204426_PROJECT/blob/main/Sprite_download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pokebase

In [ ]:
import io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pokebase as pb
from tqdm import tqdm

In [48]:
sprite_rgba = []
for i in tqdm(range(1,802)):
    source = pb.SpriteResource('pokemon', i, official_artwork=True)
    binary = source.img_data
    img = plt.imread(io.BytesIO(binary))
    img = np.asarray(img)
    sprite_rgba.append(img)

sprite_rgba = np.asarray(sprite_rgba)
sprite_rgba.shape # pokedex n == index n-1

100%|██████████| 801/801 [00:00<00:00, 1686.91it/s]


(801, 96, 96, 4)

In [49]:
from skimage.color import rgba2rgb

sprite_rgb = [rgba2rgb(img_rgba) for img_rgba in tqdm(sprite_rgba)]
sprite_rgb = np.asarray(sprite_rgb)
sprite_rgb.shape # pokedex n == index n-1

100%|██████████| 801/801 [00:00<00:00, 4095.43it/s]


(801, 96, 96, 3)

# Feature Extraction by Histogram of Orientation Gradient (HOG)
> https://www.analyticsvidhya.com/blog/2019/09/feature-engineering-images-introduction-hog-feature-descriptor/

In [54]:
from skimage.feature import hog

def feature_extraction_hog(img):
    fd, hog_image = hog(img, orientations=9, pixels_per_cell=(12,12), 
                    cells_per_block=(2, 2), visualize=True, multichannel=True)
    return fd

sprite_hog = [feature_extraction_hog(img_rgb) for img_rgb in tqdm(sprite_rgb)]
sprite_hog = np.asarray(sprite_hog)
sprite_hog.shape # pokedex n == index n-1

100%|██████████| 801/801 [00:10<00:00, 73.77it/s]


(801, 1764)

# Feature Extraction by Color Histogram
> https://www.etutorialspoint.com/index.php/310-python-opencv-histograms-of-color-image

In [66]:
import cv2 as cv

def color_histogram(img_rgba, bins=32):
    img = (img_rgba * 255).astype('uint8')
    mask = img[...,3]
    hist1 = cv.calcHist([img],[0],mask,[bins],[0,256])
    hist2 = cv.calcHist([img],[1],mask,[bins],[0,256])
    hist3 = cv.calcHist([img],[2],mask,[bins],[0,256])

    return np.r_[hist1, hist2, hist3].flatten() / hist1.sum()
    
sprite_hist = [color_histogram(img_rgba) for img_rgba in tqdm(sprite_rgba)]
sprite_hist = np.asarray(sprite_hist)
sprite_hist.shape # pokedex n == index n-1

100%|██████████| 801/801 [00:00<00:00, 5701.49it/s]


(801, 96)

In [67]:
feature_vectors = np.c_[sprite_hog, sprite_hist]
feature_vectors.shape

(801, 1860)

In [71]:
import pandas as pd

df = pd.DataFrame(feature_vectors)
df.to_csv('feature_vectors.csv', index=False)

In [74]:
!wget -O Pokemon_n.csv \
    https://raw.githubusercontent.com/tackulus/204426_PROJECT/main/Pokemon_n.csv
!wget -O feature_vectors.csv \
    https://raw.githubusercontent.com/tackulus/204426_PROJECT/main/feature_vectors.csv

--2021-10-09 10:20:06--  https://raw.githubusercontent.com/tackulus/204426_PROJECT/main/Pokemon_n.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 208753 (204K) [text/plain]
Saving to: ‘Pokemon_n.csv’

Pokemon_n.csv       100%[===================>] 203.86K  --.-KB/s    in 0.03s   

2021-10-09 10:20:06 (7.81 MB/s) - ‘Pokemon_n.csv’ saved [208753/208753]

--2021-10-09 10:20:06--  https://raw.githubusercontent.com/tackulus/204426_PROJECT/main/feature_vectors.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14693717 (14M) [text/

In [76]:
pokemon

,Unnamed: 0,pokedex_number,name,japanese_name,hp,attack,defense,sp_attack,sp_defense,speed,weight_kg,height_m,percentage_male,experience_growth,capture_rate,classfication,base_egg_steps,base_happiness,base_total,abilities,against_bug,against_dark,against_dragon,against_electric,against_fairy,against_fight,against_fire,against_flying,against_ghost,against_grass,against_ground,against_ice,against_normal,against_poison,against_psychic,against_rock,against_steel,against_water,generation,is_legendary,bug,dark,dragon,electric,fairy,fighting,fire,flying,ghost,grass,ground,ice,normal,poison,psychic,rock,steel,water
0,0,1,Bulbasaur,Fushigidaneフシギダネ,45,49,49,65,65,45,6.9,0.7,88.1,1059860,45,Seed Pokémon,5120,70,318,"['Overgrow', 'Chlorophyll']",1.00,1.0,1.0,0.5,0.5,0.5,2.0,2.0,1.0,0.25,1.0,2.0,1.0,1.0,2.0,1.0,1.0,0.5,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
1,1,2,Ivysaur,Fushigisouフシギソウ,60,62,63,80,80,60,13.0,1.0,88.1,1059860,45,Seed Pokémon,5120,70,405,"['Overgrow', 'Chlorophyll']",1.00,1.0,1.0,0.5,0.5,0.5,2.0,2.0,1.0,0.25,1.0,2.0,1.0,1.0,2.0,1.0,1.0,0.5,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
2,2,3,Venusaur,Fushigibanaフシギバナ,80,100,123,122,120,80,100.0,2.0,88.1,1059860,45,Seed Pokémon,5120,70,625,"['Overgrow', 'Chlorophyll']",1.00,1.0,1.0,0.5,0.5,0.5,2.0,2.0,1.0,0.25,1.0,2.0,1.0,1.0,2.0,1.0,1.0,0.5,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
3,3,4,Charmander,Hitokageヒトカゲ,39,52,43,60,50,65,8.5,0.6,88.1,1059860,45,Lizard Pokémon,5120,70,309,"['Blaze', 'Solar Power']",0.50,1.0,1.0,1.0,0.5,1.0,0.5,1.0,1.0,0.50,2.0,0.5,1.0,1.0,1.0,2.0,0.5,2.0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,4,5,Charmeleon,Lizardoリザード,58,64,58,80,65,80,19.0,1.1,88.1,1059860,45,Flame Pokémon,5120,70,405,"['Blaze', 'Solar Power']",0.50,1.0,1.0,1.0,0.5,1.0,0.5,1.0,1.0,0.50,2.0,0.5,1.0,1.0,1.0,2.0,0.5,2.0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
796,796,797,Celesteela,Tekkaguyaテッカグヤ,97,101,103,107,101,61,999.9,9.2,0.0,1250000,25,Launch Pokémon,30720,0,570,['Beast Boost'],0.25,1.0,0.5,2.0,0.5,1.0,2.0,0.5,1.0,0.25,0.0,1.0,0.5,0.0,0.5,1.0,0.5,1.0,7,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
797,797,798,Kartana,Kamiturugiカミツルギ,59,181,131,59,31,109,0.1,0.3,0.0,1250000,255,Drawn Sword Pokémon,30720,0,570,['Beast Boost'],1.00,1.0,0.5,0.5,0.5,2.0,4.0,1.0,1.0,0.25,1.0,1.0,0.5,0.0,0.5,0.5,0.5,0.5,7,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0
798,798,799,Guzzlord,Akuzikingアクジキング,223,101,53,97,53,43,888.0,5.5,0.0,1250000,15,Junkivore Pokémon,30720,0,570,['Beast Boost'],2.00,0.5,2.0,0.5,4.0,2.0,0.5,1.0,0.5,0.50,1.0,2.0,1.0,1.0,0.0,1.0,1.0,0.5,7,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
799,799,800,Necrozma,Necrozmaネクロズマ,97,107,101,127,89,79,230.0,2.4,0.0,1250000,3,Prism Pokémon,30720,0,600,['Prism Armor'],2.00,2.0,1.0,1.0,1.0,0.5,1.0,1.0,2.0,1.00,1.0,1.0,1.0,1.0,0.5,1.0,1.0,1.0,7,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [79]:
pokemon = pd.read_csv('Pokemon_n.csv').iloc[:,1:]
feature_vectors = pd.read_csv('feature_vectors.csv')

df = pd.concat([pokemon, feature_vectors], axis=1)
df.to_csv('ALL.csv', index=False)